# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## All CpG

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\Male\GeneCpG\Cognition
df = pd.read_csv('CognitionCateg_Methylome_Male.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\NonMotors\Derived250\Male\GeneCpG\Cognition
(307, 123024)


,Unnamed: 0,Sentrix,PATNO,HYS,Gender,APPRDX,Montreal_Cognitive,cg08012045,cg08300930,cg00134295,...,cg10054014,cg15364874,cg06622006,cg22071811,cg24960706,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,0,200991620021_R01C01,3001,2,1,1,0,0.962437,0.964442,0.243530,...,0.852622,0.880126,0.876185,0.770531,0.830047,0.216268,0.234350,0.681550,0.103758,0.837890
1,1,200973410167_R04C01,3007,2,1,1,0,0.962158,0.963041,0.226060,...,0.900772,0.901686,0.906222,0.804494,0.905520,0.254265,0.290337,0.760554,0.075480,0.833824
2,2,200973410167_R05C01,3010,2,1,1,0,0.958309,0.964922,0.192857,...,0.904735,0.898241,0.914635,0.819989,0.897821,0.229338,0.232169,0.775542,0.119109,0.846339


In [3]:
df1 = df.drop(['Unnamed: 0','Sentrix','PATNO','HYS','Gender','Montreal_Cognitive'], axis=1)
df1.head(2)

,APPRDX,cg08012045,cg08300930,cg00134295,cg10537824,cg04990650,cg03379001,cg02215627,cg06699201,cg18100935,...,cg10054014,cg15364874,cg06622006,cg22071811,cg24960706,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,1,0.962437,0.964442,0.24353,0.826172,0.845330,0.852316,0.871498,0.653140,0.89167,...,0.852622,0.880126,0.876185,0.770531,0.830047,0.216268,0.234350,0.681550,0.103758,0.837890
1,1,0.962158,0.963041,0.22606,0.729373,0.830303,0.764861,0.821899,0.717675,0.88783,...,0.900772,0.901686,0.906222,0.804494,0.905520,0.254265,0.290337,0.760554,0.075480,0.833824


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(50))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.45
2
Mean accuracy: 0.5399999999999999
3
Mean accuracy: 0.45999999999999996
4
Mean accuracy: 0.53
5
Mean accuracy: 0.42000000000000004
6
Mean accuracy: 0.51
7
Mean accuracy: 0.48999999999999994
8
Mean accuracy: 0.57
9
Mean accuracy: 0.51
10
Mean accuracy: 0.56
11
Mean accuracy: 0.5700000000000001
12
Mean accuracy: 0.56
13
Mean accuracy: 0.43000000000000005
14
Mean accuracy: 0.5700000000000001
15
Mean accuracy: 0.52
16
Mean accuracy: 0.4600000000000001
17
Mean accuracy: 0.41
18
Mean accuracy: 0.48999999999999994
19
Mean accuracy: 0.55
20
Mean accuracy: 0.47000000000000003
21
Mean accuracy: 0.5700000000000001
22
Mean accuracy: 0.5200000000000001
23
Mean accuracy: 0.53
24
Mean accuracy: 0.6300000000000001
25
Mean accuracy: 0.53
26
Mean accuracy: 0.45
27
Mean accuracy: 0.41999999999999993
28
Mean accuracy: 0.52
29
Mean accuracy: 0.45999999999999996
30
Mean accuracy: 0.6099999999999999
31
Mean accuracy: 0.4600000000000001
32
Mean accuracy: 0.54
33
Mean accuracy: 0.4700000

In [5]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_AllCpG_Male_100RF.csv", index = False)

0.5077999999999999


## RanFor 1000 times. Samples = 50

In [6]:
df2 = df1.drop (['APPRDX'], axis=1)

In [7]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(50))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.48
2
Mean accuracy: 0.45
3
Mean accuracy: 0.49000000000000005
4
Mean accuracy: 0.44000000000000006
5
Mean accuracy: 0.5199999999999999
6
Mean accuracy: 0.51
7
Mean accuracy: 0.4999999999999999
8
Mean accuracy: 0.63
9
Mean accuracy: 0.45
10
Mean accuracy: 0.49000000000000005
11
Mean accuracy: 0.38
12
Mean accuracy: 0.51
13
Mean accuracy: 0.41
14
Mean accuracy: 0.43000000000000005
15
Mean accuracy: 0.4600000000000001
16
Mean accuracy: 0.39
17
Mean accuracy: 0.48999999999999994
18
Mean accuracy: 0.5399999999999999
19
Mean accuracy: 0.62
20
Mean accuracy: 0.44000000000000006
21
Mean accuracy: 0.5700000000000001
22
Mean accuracy: 0.5
23
Mean accuracy: 0.51
24
Mean accuracy: 0.47000000000000003
25
Mean accuracy: 0.48
26
Mean accuracy: 0.41
27
Mean accuracy: 0.5599999999999999
28
Mean accuracy: 0.5900000000000001
29
Mean accuracy: 0.53
30
Mean accuracy: 0.5399999999999999
31
Mean accuracy: 0.5700000000000001
32
Mean accuracy: 0.39
33
Mean accuracy: 0.4600000000000001
34
Mea

In [8]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_Rand50CpG_Male_100RF.csv", index = False)

0.493


## RanFor 100 times. Samples = 99

In [9]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(99, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(50))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.39
2
Mean accuracy: 0.4600000000000001
3
Mean accuracy: 0.6199999999999999
4
Mean accuracy: 0.45999999999999996
5
Mean accuracy: 0.52
6
Mean accuracy: 0.51
7
Mean accuracy: 0.53
8
Mean accuracy: 0.5900000000000001
9
Mean accuracy: 0.48
10
Mean accuracy: 0.5
11
Mean accuracy: 0.43
12
Mean accuracy: 0.47000000000000003
13
Mean accuracy: 0.5
14
Mean accuracy: 0.48999999999999994
15
Mean accuracy: 0.45
16
Mean accuracy: 0.41
17
Mean accuracy: 0.52
18
Mean accuracy: 0.4600000000000001
19
Mean accuracy: 0.48999999999999994
20
Mean accuracy: 0.47000000000000003
21
Mean accuracy: 0.59
22
Mean accuracy: 0.4600000000000001
23
Mean accuracy: 0.5599999999999999
24
Mean accuracy: 0.44000000000000006
25
Mean accuracy: 0.4699999999999999
26
Mean accuracy: 0.38999999999999996
27
Mean accuracy: 0.57
28
Mean accuracy: 0.6100000000000001
29
Mean accuracy: 0.44000000000000006
30
Mean accuracy: 0.49000000000000005
31
Mean accuracy: 0.5599999999999999
32
Mean accuracy: 0.48
33
Mean accura

In [10]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_Rand98CpG_Male_100RF.csv", index = False)

0.49599999999999994


## RanFor Top 50 Boruta 100 times. Samples = 50

In [11]:
df = pd.read_csv('Cognition_Boruta_Top50_Data_Male.csv')
print(df.shape)
df.head(2)

(307, 52)


,PATNO,APPRDX,cg27171575,cg10693616,cg06098693,cg16444863,cg18174834,cg06129513,cg13312668,cg07456278,...,cg20414811,cg22216017,cg15809230,cg17624978,cg16606731,cg13468400,cg11755107,cg10121816,cg13240962,cg15419852
0,3001,1,0.853809,0.012194,0.910787,0.931500,0.040499,0.843875,0.858468,0.582067,...,0.833260,0.043279,0.015557,0.866434,0.574604,0.812987,0.014287,0.011981,0.778440,0.025940
1,3007,1,0.895204,0.012696,0.924330,0.937715,0.034218,0.920245,0.895692,0.667340,...,0.873684,0.045068,0.015942,0.899224,0.625920,0.897971,0.015179,0.014084,0.799525,0.037032


In [12]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg27171575,cg10693616,cg06098693,cg16444863,cg18174834,cg06129513,cg13312668,cg07456278,cg05817202,...,cg20414811,cg22216017,cg15809230,cg17624978,cg16606731,cg13468400,cg11755107,cg10121816,cg13240962,cg15419852
0,1,0.853809,0.012194,0.910787,0.931500,0.040499,0.843875,0.858468,0.582067,0.823899,...,0.833260,0.043279,0.015557,0.866434,0.574604,0.812987,0.014287,0.011981,0.778440,0.025940
1,1,0.895204,0.012696,0.924330,0.937715,0.034218,0.920245,0.895692,0.667340,0.849287,...,0.873684,0.045068,0.015942,0.899224,0.625920,0.897971,0.015179,0.014084,0.799525,0.037032


In [13]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(50))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.6000000000000001
2
Mean accuracy: 0.6799999999999999
3
Mean accuracy: 0.73
4
Mean accuracy: 0.72
5
Mean accuracy: 0.6799999999999999
6
Mean accuracy: 0.6699999999999999
7
Mean accuracy: 0.67
8
Mean accuracy: 0.7100000000000001
9
Mean accuracy: 0.74
10
Mean accuracy: 0.69
11
Mean accuracy: 0.6599999999999999
12
Mean accuracy: 0.65
13
Mean accuracy: 0.7300000000000001
14
Mean accuracy: 0.72
15
Mean accuracy: 0.6399999999999999
16
Mean accuracy: 0.6599999999999999
17
Mean accuracy: 0.6100000000000001
18
Mean accuracy: 0.72
19
Mean accuracy: 0.6
20
Mean accuracy: 0.71
21
Mean accuracy: 0.71
22
Mean accuracy: 0.6300000000000001
23
Mean accuracy: 0.7500000000000001
24
Mean accuracy: 0.73
25
Mean accuracy: 0.6300000000000001
26
Mean accuracy: 0.6499999999999999
27
Mean accuracy: 0.7000000000000001
28
Mean accuracy: 0.74
29
Mean accuracy: 0.58
30
Mean accuracy: 0.71
31
Mean accuracy: 0.67
32
Mean accuracy: 0.7
33
Mean accuracy: 0.6300000000000001
34
Mean accuracy: 0.63
35
Me

In [14]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_Boruta50CpG_Male_100RF.csv", index = False)

0.6733


## RanFor Top 50 NMI 100 times. Samples = 50

In [15]:
df = pd.read_csv('Cognition_NMI_Top50_Male_Data.csv')
print(df.shape)
df.head(2)

(307, 51)


,APPRDX,cg03787486,cg01360545,cg12573516,cg18859045,cg11450827,cg20981182,cg19505862,cg26941073,cg22917487,...,cg19711613,cg10131227,cg15469205,cg12613770,cg20003494,cg15736994,cg16840743,cg03909844,cg00558297,cg17668069
0,1,0.035693,0.605723,0.026891,0.233609,0.797882,0.070829,0.040946,0.925943,0.875027,...,0.841310,0.929469,0.865868,0.838669,0.025708,0.523965,0.072730,0.675087,0.776180,0.044870
1,1,0.034595,0.633931,0.050838,0.335034,0.805798,0.079024,0.040822,0.915694,0.851476,...,0.839718,0.719363,0.830760,0.848701,0.045513,0.562597,0.072311,0.769234,0.366488,0.045872


In [16]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df.groupby('APPRDX').apply(lambda x: x.sample(50))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.6199999999999999
2
Mean accuracy: 0.68
3
Mean accuracy: 0.71
4
Mean accuracy: 0.6599999999999999
5
Mean accuracy: 0.7
6
Mean accuracy: 0.72
7
Mean accuracy: 0.67
8
Mean accuracy: 0.73
9
Mean accuracy: 0.73
10
Mean accuracy: 0.6199999999999999
11
Mean accuracy: 0.74
12
Mean accuracy: 0.72
13
Mean accuracy: 0.7
14
Mean accuracy: 0.7499999999999999
15
Mean accuracy: 0.6399999999999999
16
Mean accuracy: 0.6200000000000001
17
Mean accuracy: 0.67
18
Mean accuracy: 0.6799999999999999
19
Mean accuracy: 0.63
20
Mean accuracy: 0.6900000000000001
21
Mean accuracy: 0.7300000000000001
22
Mean accuracy: 0.73
23
Mean accuracy: 0.7100000000000001
24
Mean accuracy: 0.76
25
Mean accuracy: 0.76
26
Mean accuracy: 0.77
27
Mean accuracy: 0.7
28
Mean accuracy: 0.7
29
Mean accuracy: 0.63
30
Mean accuracy: 0.71
31
Mean accuracy: 0.7100000000000001
32
Mean accuracy: 0.7300000000000001
33
Mean accuracy: 0.72
34
Mean accuracy: 0.7300000000000001
35
Mean accuracy: 0.6699999999999999
36
Mean accu

In [17]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_NMI50CpG_Male_100RF.csv", index = False)

0.6987999999999996


## RanFor Top 50 Boruta + NMI 100 times. Samples = 99

In [18]:
df = pd.read_csv('Cognition_Top50sMerge_Data_Male.csv')
print(df.shape)
df.head(2)

(307, 101)


,PATNO,APPRDX,cg27171575,cg10693616,cg06098693,cg16444863,cg18174834,cg06129513,cg13312668,cg07456278,...,cg19711613,cg10131227,cg15469205,cg12613770,cg20003494,cg15736994,cg16840743,cg03909844,cg00558297,cg17668069
0,3001,1,0.853809,0.012194,0.910787,0.931500,0.040499,0.843875,0.858468,0.582067,...,0.841310,0.929469,0.865868,0.838669,0.025708,0.523965,0.072730,0.675087,0.776180,0.044870
1,3007,1,0.895204,0.012696,0.924330,0.937715,0.034218,0.920245,0.895692,0.667340,...,0.839718,0.719363,0.830760,0.848701,0.045513,0.562597,0.072311,0.769234,0.366488,0.045872


In [19]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg27171575,cg10693616,cg06098693,cg16444863,cg18174834,cg06129513,cg13312668,cg07456278,cg05817202,...,cg19711613,cg10131227,cg15469205,cg12613770,cg20003494,cg15736994,cg16840743,cg03909844,cg00558297,cg17668069
0,1,0.853809,0.012194,0.910787,0.931500,0.040499,0.843875,0.858468,0.582067,0.823899,...,0.841310,0.929469,0.865868,0.838669,0.025708,0.523965,0.072730,0.675087,0.776180,0.044870
1,1,0.895204,0.012696,0.924330,0.937715,0.034218,0.920245,0.895692,0.667340,0.849287,...,0.839718,0.719363,0.830760,0.848701,0.045513,0.562597,0.072311,0.769234,0.366488,0.045872


In [20]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(50))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.6500000000000001
2
Mean accuracy: 0.7499999999999999
3
Mean accuracy: 0.7
4
Mean accuracy: 0.7299999999999999
5
Mean accuracy: 0.75
6
Mean accuracy: 0.72
7
Mean accuracy: 0.6199999999999999
8
Mean accuracy: 0.73
9
Mean accuracy: 0.75
10
Mean accuracy: 0.6900000000000001
11
Mean accuracy: 0.7400000000000001
12
Mean accuracy: 0.7100000000000001
13
Mean accuracy: 0.76
14
Mean accuracy: 0.79
15
Mean accuracy: 0.7
16
Mean accuracy: 0.64
17
Mean accuracy: 0.63
18
Mean accuracy: 0.6799999999999999
19
Mean accuracy: 0.6599999999999999
20
Mean accuracy: 0.77
21
Mean accuracy: 0.73
22
Mean accuracy: 0.74
23
Mean accuracy: 0.75
24
Mean accuracy: 0.75
25
Mean accuracy: 0.7
26
Mean accuracy: 0.78
27
Mean accuracy: 0.73
28
Mean accuracy: 0.69
29
Mean accuracy: 0.67
30
Mean accuracy: 0.76
31
Mean accuracy: 0.72
32
Mean accuracy: 0.7500000000000001
33
Mean accuracy: 0.74
34
Mean accuracy: 0.74
35
Mean accuracy: 0.71
36
Mean accuracy: 0.66
37
Mean accuracy: 0.75
38
Mean accuracy: 0.6

In [21]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_Top50sCpG99_Male_100RF.csv", index = False)

0.7145999999999999
